In [6]:
import pandas as pd
import geopandas as gpd
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import os

os.getcwd()

'/Users/tomokitakata/Desktop/research/dev/muldistribution/models'

# データ読み込みと前処理

In [ ]:
# 医療機関POIデータ読み込み
rihabili = gpd.read_file('../data/medicalpoi/2010/リハビリテーション科.shp')
gastro = gpd.read_file('../data/medicalpoi/2010/消化器内科.shp')
cardio = gpd.read_file('../data/medicalpoi/2010/循環器内科.shp')
uro = gpd.read_file('../data/medicalpoi/2010/泌尿器内科.shp')
# gridデータ読み込み
grid = gpd.read_file('../data/grid/wholejapan_grid/wholejapan_grid.shp')
# 医療機関POIデータが各gridに何件あるかを集計
# gpkgデータとして出力

In [7]:
import geopandas as gpd
from pathlib import Path

# ── 0. パス設定 ─────────────────────────────────
poi_files = {
    "Rehabilitation": "../data/medicalpoi/2010/リハビリテーション科.shp",
    "Gastro":         "../data/medicalpoi/2010/消化器内科.shp",
    "Cardio":         "../data/medicalpoi/2010/循環器内科.shp",
    "Urology":        "../data/medicalpoi/2010/泌尿器内科.shp",
}
grid_path = "../data/grid/wholejapan_grid/wholejapan_grid.shp"
out_gpkg  = Path("grid_with_poi_counts.gpkg")

# ── 1. グリッド読み込み（基準 CRS を決める） ──────────
grid = gpd.read_file(grid_path)
grid = grid.set_crs(4326) if grid.crs is None else grid   # ↓ポイント類もこの座標系に合わせる

# ユニークキー列（無ければ index を使う）
gid_col = "grid_id"
if gid_col not in grid.columns:
    grid[gid_col] = grid.index.astype(int)

# ── 2. 各診療科について「グリッド内件数」を計算 ───────
for eng, path in poi_files.items():
    poi = gpd.read_file(path)
    poi = poi.set_crs(4326) if poi.crs is None else poi     # 度数系と仮定
    poi = poi.to_crs(grid.crs)                              # CRS 揃える

    # a) 空間結合（各ポイントが属するグリッド ID を取る）
    joined = gpd.sjoin(poi[[poi.geometry.name]],
                       grid[[gid_col, grid.geometry.name]],
                       predicate="within",
                       how="left")

    # b) グリッド単位で件数カウント
    cnt = (
        joined
        .groupby(gid_col)
        .size()
        .rename(f"{eng}_cnt")        # 列名例: Rehabilitation_cnt
        .to_frame()
    )

    # c) グリッド本体にマージ（無いところは 0）
    grid = grid.merge(cnt, on=gid_col, how="left")
    grid[f"{eng}_cnt"] = grid[f"{eng}_cnt"].fillna(0).astype(int)

# ── 3. GeoPackage に書き出し  (全診療科まとめて 1 レイヤ) ─
out_gpkg.unlink(missing_ok=True)        # 既存を削除
grid.to_file(out_gpkg, layer="grid_poi_counts", driver="GPKG")
print("✅ 書き出し完了 →", out_gpkg.resolve())

✅ 書き出し完了 → /Users/tomokitakata/Desktop/research/dev/muldistribution/models/grid_with_poi_counts.gpkg


In [11]:
"""
make_ml_table.py
────────────────────────────────────────────────────────
1. 日本全国メッシュ (wholejapan_grid.shp) を基盤に
2. 各診療科 POI 件数を集計           → *_cnt 列
3. 人口メッシュ (pop_grid.gpkg) を合体 → population 列
4. 機械学習用の DataFrame (= geometry なし) を保存
   ・grid_id, centroid_X/Y, 各 *_cnt, population
────────────────────────────────────────────────────────
"""

from pathlib import Path
import geopandas as gpd
import pandas as pd

# ─── 入出力パス ─────────────────────────────────────────
grid_path = Path("../data/grid/wholejapan_grid/wholejapan_grid.shp")
pop_path  = Path("../data/pop/2010/pop_grid.gpkg")           # ★人口
poi_files = {
    "Rehabilitation": "../data/medicalpoi/2010/リハビリテーション科.shp",
    "Gastro":         "../data/medicalpoi/2010/消化器内科.shp",
    "Cardio":         "../data/medicalpoi/2010/循環器内科.shp",
    "Urology":        "../data/medicalpoi/2010/泌尿器内科.shp",
}

out_geo = Path("grid_with_features.gpkg")   # 中間 GeoPackage
out_df  = Path("ml_table.parquet")          # 最終 ML テーブル

pop_col = "Total population"               # 人口列名 in pop_grid.gpkg
gid_col = "grid_id"                        # → なければ index で作成

# ─── 1. 基盤グリッド読み込み ────────────────────────────

# 変更後（存在すればそのまま、無ければ 4326 をセット）
grid = gpd.read_file(grid_path)
grid = grid if grid.crs is not None else grid.set_crs(4326)
if gid_col not in grid.columns:
    grid[gid_col] = grid.index.astype(int)

# ─── 2. 各診療科の POI → グリッド件数集計 ───────────────
for name, path in poi_files.items():
    poi = gpd.read_file(path).set_crs(4326).to_crs(grid.crs)
    join = gpd.sjoin(poi[[poi.geometry.name]],
                     grid[[gid_col, grid.geometry.name]],
                     predicate="within", how="left")
    cnt  = (
        join.groupby(gid_col).size()
             .rename(f"{name}_cnt")
             .to_frame()
    )
    grid = grid.merge(cnt, on=gid_col, how="left")
    grid[f"{name}_cnt"] = grid[f"{name}_cnt"].fillna(0).astype("uint16")

# ─── 3. 人口メッシュの結合 ──────────────────────────────
pop = gpd.read_file(pop_path)            # 読み込み
if pop.crs is None:                      # ← CRS が無い時だけラベル付与
    pop = pop.set_crs(4326)
pop = pop.to_crs(grid.crs)               
if pop_col not in pop.columns:
    raise KeyError(f"列 {pop_col} が見当たりません")

# → geometry 同士で重ね合わせ（intersects で十分）
pop_trim = pop[[pop_col, pop.geometry.name]]
join_pop = gpd.sjoin(pop_trim, grid[[gid_col, grid.geometry.name]],
                     predicate="intersects", how="left")

pop_sum = (
    join_pop.groupby(gid_col)[pop_col].sum()
            .rename("population")
            .to_frame()
)

grid = grid.merge(pop_sum, on=gid_col, how="left")
grid["population"] = grid["population"].fillna(0).astype("uint32")

# ─── 4. セントロイド座標列を追加 (ML 用) ────────────────
# 投影系 (例: EPSG:6697 = JGD2011 / Albers) へ一時変換して計算
proj_epsg = 6697
grid_metric = grid.to_crs(proj_epsg)
cent        = grid_metric.geometry.centroid   # ←メートル系なら警告無し
grid["centroid_X"] = cent.x
grid["centroid_Y"] = cent.y

# ─── 5. 保存 ───────────────────────────────────────────
#   a) GeoPackage (QGIS で確認用)
out_geo.unlink(missing_ok=True)
grid.to_file(out_geo, layer="grid_features", driver="GPKG")
print("✅ GeoPackage:", out_geo.resolve())

#   b) 機械学習テーブル (geometry を除く)
df = grid.drop(columns="geometry")
df.to_csv("ml_table.csv", index=False)

/var/folders/nw/cvnq7pwd21qdy7ff97bszbq40000gn/T/ipykernel_73973/1527567897.py:80: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent        = grid_metric.geometry.centroid   # ←メートル系なら警告無し


✅ GeoPackage: /Users/tomokitakata/Desktop/research/dev/muldistribution/models/grid_with_features.gpkg


In [12]:
df.head()

,KEY_CODE,MESH1_ID,MESH2_ID,MESH3_ID,OBJ_ID,grid_id,Rehabilitation_cnt,Gastro_cnt,Cardio_cnt,Urology_cnt,population,centroid_X,centroid_Y
0,66440000,6644,00,00,1.0,0,0,0,0,0,4,144.00625,44.004167
1,66440001,6644,00,01,2.0,1,0,0,0,0,0,144.01875,44.004167
2,66440002,6644,00,02,3.0,2,0,0,0,0,0,144.03125,44.004167
3,66440003,6644,00,03,4.0,3,0,0,0,0,0,144.04375,44.004167
4,66440004,6644,00,04,5.0,4,0,0,0,0,0,144.05625,44.004167


# random forest の構築

In [14]:
# rf_gridsearch_by_section.py
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.utils import parallel_backend
from tqdm  import tqdm

# ── 0.  入力ファイル ────────────────────────────────
df_path = Path("ml_table.csv")          # 作成済み CSV
df = pd.read_csv(df_path)

# ── 1.  共通設定 ──────────────────────────────────
features = ["population", "centroid_X", "centroid_Y"]
targets  = {
    "Rehabilitation_cnt": "Rehabilitation",
    "Gastro_cnt":         "Gastroenterology",
    "Cardio_cnt":         "Cardiology",
    "Urology_cnt":        "Urology (Med.)"
}
test_size    = 0.2
random_state = 42
cv_folds     = 5

# RMSE（正の値）を返す scorer
rmse_scorer = make_scorer(
    lambda y, yhat: np.sqrt(mean_squared_error(y, yhat)),
    greater_is_better=False       # GridSearch は「大きいほど良い」を期待
)

# 代表的なハイパーパラメータセット
param_grid = {
    "n_estimators":      [200, 400, 800],
    "max_depth":         [None, 15, 30, 60],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf":  [1, 2, 4],
    "max_features":      ["sqrt", "log2", 0.6]   # 0.6 = 全特徴量の60%
}

print(f"─ Grid search: {len(param_grid['n_estimators'])*len(param_grid['max_depth'])*len(param_grid['min_samples_split'])*len(param_grid['min_samples_leaf'])*len(param_grid['max_features'])} parameter combos × {cv_folds}-fold CV\n")

# ── 2.  診療科ごとに学習 ───────────────────────────
for col, label in tqdm(targets.items()):
    X = df[features]
    y = df[col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    base_rf = RandomForestRegressor(random_state=random_state)

    gsearch = GridSearchCV(
        base_rf,
        param_grid,
        scoring={"RMSE": rmse_scorer, "R2": "r2"},
        refit="RMSE",         # = 最小 RMSE を選ぶ
        cv=cv_folds,
        n_jobs=-1,            # CPU 全コア
        verbose=0
    )

    # scikit‑learn 1.4 以降は loky backend がデフォルト
    with parallel_backend("loky"):
        gsearch.fit(X_train, y_train)

    best = gsearch.best_estimator_
    y_pred = best.predict(X_test)

    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test   = r2_score(y_test, y_pred)

    print(f"{label:20s} |  RMSE_test = {rmse_test:7.3f}   R²_test = {r2_test:6.3f}")
    print("   best_params :", gsearch.best_params_, "\n")

─ Grid search: 324 parameter combos × 5-fold CV



  0%|          | 0/4 [00:00<?, ?it/s]/Users/tomokitakata/Desktop/research/dev/muldistribution/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:71: FutureWarning: Class parallel_backend is deprecated; deprecated in 1.5 to be removed in 1.7. Use joblib.{} instead.
  warnings.warn(msg, category=FutureWarning)
/Users/tomokitakata/Desktop/research/dev/muldistribution/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
  0%|          | 0/4 [3:55:39<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
def train_per_department(df, input_columns, target_columns, model_class=RandomForestRegressor, model_params=None):
    results = {}
    for col in tqdm(target_columns):
        print(f"\n==== 🏥 {col} の学習開始 ====")
        X = df[input_columns]
        y = df[col]

        # 欠損除去
        data = pd.concat([X, y], axis=1).fillna(0)
        X = data[input_columns]
        y = data[col]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = model_class(**(model_params or {}))
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"  RMSE: {rmse:.3f}, R²: {r2:.3f}")

        results[col] = {
            "model": model,
            "rmse": rmse,
            "r2": r2
        }
    return results

In [17]:
input_cols = ["population"]
target_cols = ["inner_med_grid"]

results = train_per_department(
    df=mesh_df,
    input_columns=input_cols,
    target_columns=target_cols,
    model_class=RandomForestRegressor,
    model_params={"n_estimators": 100}
)


==== 🏥 inner_med_grid の学習開始 ====
  RMSE: 0.413, R²: 0.462


In [34]:
def evaluate_models(df, input_columns, target_column,
                    rf_params=None, xgb_params=None, lgbm_params=None,
                    test_size=0.2, random_state=42):
    # 入力と出力
    X = df[input_columns].fillna(0)
    y = df[target_column].fillna(0)

    # データ分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # yが2次元(n_samples, 1)になっちゃうので、1次元(n_samples,)に変換
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()

    # モデルとパラメータ
    models = {
        "RandomForest": RandomForestRegressor(random_state=random_state, **(rf_params or {})),
        "XGBoost": XGBRegressor(random_state=random_state, verbosity=0, **(xgb_params or {})),
        "LightGBM": LGBMRegressor(random_state=random_state, **(lgbm_params or {}))
    }

    results = {}
    records = []



    # 各モデルの学習・予測・評価
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {
            "model": model,
            "rmse": rmse,
            "r2": r2
        }
        records.append({
            "model": name,
            "rmse": rmse,
            "r2": r2
        })

        print(f"[{name}] RMSE: {rmse:.3f}, R²: {r2:.3f}")

    # 結果をDataFrame化
    result_df = pd.DataFrame(records)

    # RMSEが最小のモデルを明示的に選ぶ（sortを使わない）
    min_rmse = result_df["rmse"].min()
    best_model_row = result_df[result_df["rmse"] == min_rmse].iloc[0]
    best_model_name = best_model_row["model"]

    print(f"best model：{best_model_name}（min RMSE: {min_rmse:.3f}）")

    # 表示・見やすさのために結果をrmse順に並び替えて返す
    result_df = result_df.sort_values("rmse").reset_index(drop=True)


    return results, result_df

In [33]:
results, result_df = evaluate_models(
    df=mesh_df,
    input_columns=["population"],
    target_column=["inner_med_grid"],
    rf_params={"n_estimators": 100},
    xgb_params={"n_estimators": 100, "learning_rate": 0.1},
    lgbm_params={"n_estimators": 100, "learning_rate": 0.1}
)

result_df

 33%|███▎      | 1/3 [00:06<00:13,  6.95s/it]

[RandomForest] RMSE: 0.415, R²: 0.461


 67%|██████▋   | 2/3 [00:07<00:03,  3.00s/it]

[XGBoost] RMSE: 0.311, R²: 0.595
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 401200, number of used features: 1
[LightGBM] [Info] Start training from score 0.118053


100%|██████████| 3/3 [00:07<00:00,  2.59s/it]

[LightGBM] RMSE: 0.303, R²: 0.606
best model：LightGBM（min RMSE: 0.303）


,model,rmse,r2
0,LightGBM,0.303173,0.605643
1,XGBoost,0.311187,0.595220
2,RandomForest,0.414521,0.460806


In [31]:
result_df.head()

,model,rmse,r2
0,LightGBM,0.303173,0.605643
1,XGBoost,0.311187,0.595220
2,RandomForest,0.414521,0.460806


In [35]:
from itertools import product

def generate_param_grid(param_options):
    keys, values = zip(*param_options.items())
    return [dict(zip(keys, v)) for v in product(*values)]

In [39]:
import time
import pandas as pd

def run_sequential_experiments_with_time_tqdm(df, input_columns, target_column,
                                              rf_grid, xgb_grid, lgbm_grid):
    all_results = []

    rf_combos = generate_param_grid(rf_grid)
    xgb_combos = generate_param_grid(xgb_grid)
    lgbm_combos = generate_param_grid(lgbm_grid)

    total_runs = len(rf_combos) + len(xgb_combos) + len(lgbm_combos)
    pbar = tqdm(total=total_runs, desc="Running experiments")

    for rf_params in rf_combos:
        start = time.time()
        results, df_result = evaluate_models(df, input_columns, target_column, rf_params=rf_params)
        duration = time.time() - start
        df_result["params"] = str(rf_params)
        df_result["model_type"] = "RandomForest"
        df_result["time_sec"] = duration
        all_results.append(df_result)
        pbar.update(1)

    for xgb_params in xgb_combos:
        start = time.time()
        results, df_result = evaluate_models(df, input_columns, target_column, xgb_params=xgb_params)
        duration = time.time() - start
        df_result["params"] = str(xgb_params)
        df_result["model_type"] = "XGBoost"
        df_result["time_sec"] = duration
        all_results.append(df_result)
        pbar.update(1)

    for lgbm_params in lgbm_combos:
        start = time.time()
        results, df_result = evaluate_models(df, input_columns, target_column, lgbm_params=lgbm_params)
        duration = time.time() - start
        df_result["params"] = str(lgbm_params)
        df_result["model_type"] = "LightGBM"
        df_result["time_sec"] = duration
        all_results.append(df_result)
        pbar.update(1)

    pbar.close()
    final_df = pd.concat(all_results, ignore_index=True)
    return final_df

In [45]:
rf_param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 3]
}

xgb_param_grid = {
    "n_estimators": [50, 100],
    "learning_rate": [0.01, 0.1, 0.3],
    "max_depth": [3, 6],
    "min_child_weight": [1, 5],
    "subsample": [0.7, 1.0],
    "colsample_bytree": [0.7, 1.0]
}

lgbm_param_grid = {
    "n_estimators": [50, 100],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [5, 10, -1],
    "num_leaves": [15, 31, 63],
    "min_child_samples": [10, 20],
    "subsample": [0.7, 1.0],
    "colsample_bytree": [0.7, 1.0]
}

# 実行
experiment_results = run_sequential_experiments_with_time_tqdm(
    df=mesh_df,
    input_columns=["population"],
    target_column="inner_med_grid",
    rf_grid=rf_param_grid,
    xgb_grid=xgb_param_grid,
    lgbm_grid=lgbm_param_grid
)

# 上位結果を確認
print(experiment_results.sort_values("rmse").head())

KeyboardInterrupt: 

In [44]:
experiment_results.sort_values("rmse")

,model,rmse,r2,params,model_type,time_sec
0,RandomForest,0.301596,0.607695,"{'n_estimators': 50, 'max_depth': 5}",RandomForest,3.060278
6,RandomForest,0.301738,0.607511,"{'n_estimators': 100, 'max_depth': 5}",RandomForest,5.166987
36,LightGBM,0.303133,0.605696,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",LightGBM,7.766892
1,LightGBM,0.303173,0.605643,"{'n_estimators': 50, 'max_depth': 5}",RandomForest,3.060278
3,LightGBM,0.303173,0.605643,"{'n_estimators': 50, 'max_depth': 10}",RandomForest,3.828346
7,LightGBM,0.303173,0.605643,"{'n_estimators': 100, 'max_depth': 5}",RandomForest,5.166987
9,LightGBM,0.303173,0.605643,"{'n_estimators': 100, 'max_depth': 10}",RandomForest,6.970121
15,LightGBM,0.303173,0.605643,"{'n_estimators': 50, 'learning_rate': 0.3, 'ma...",XGBoost,7.800660
12,LightGBM,0.303173,0.605643,"{'n_estimators': 50, 'learning_rate': 0.1, 'ma...",XGBoost,7.716852
39,LightGBM,0.303175,0.605641,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",LightGBM,7.977079
